In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import scipy.sparse as sp
import time
import argparse
import numpy as np
import easydict
import math
# from .layers import *
# from .models import *
# from .utils import *
from __future__ import division
from __future__ import print_function
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from setuptools import setup
from setuptools import find_packages
from pygcn.utils import load_data, accuracy
from pygcn.models import GCN
from pygcn.layers import GraphConvolution

In [49]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_data(path="C:/Users/chaeny/Desktop/DILab/Explainable AI/gcn/data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [50]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [51]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [52]:
# Training settings
args = easydict.EasyDict({
    "no-cuda": False,
    "fastmode": False,
    "seed": 42,
    "epoch": 200,
    "lr": 0.01,
    "weight_decay": 5e-4,
    "hidden": 16,
    "dropout": 0.5
})
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

In [53]:
# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

Loading cora dataset...


In [59]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss train: {:.4f}'.format(loss_train.item()),
          'acc train: {:.4f}'.format(acc_train.item()),
          'loss val: {:.4f}'.format(loss_val.item()),
          'acc val: {:.4f}'.format(acc_val.item()))

In [60]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [61]:
# Train model
t_total = time.time()
for epoch in range(args.epoch):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss train: 0.4641 acc train: 0.9143 loss val: 0.7248 acc val: 0.7900
Epoch: 0002 loss train: 0.4231 acc train: 0.9357 loss val: 0.7244 acc val: 0.7900
Epoch: 0003 loss train: 0.4060 acc train: 0.9357 loss val: 0.7234 acc val: 0.7900
Epoch: 0004 loss train: 0.4054 acc train: 0.9571 loss val: 0.7207 acc val: 0.7900
Epoch: 0005 loss train: 0.3750 acc train: 0.9643 loss val: 0.7171 acc val: 0.7900
Epoch: 0006 loss train: 0.4170 acc train: 0.9357 loss val: 0.7131 acc val: 0.7933
Epoch: 0007 loss train: 0.4571 acc train: 0.9429 loss val: 0.7098 acc val: 0.8000
Epoch: 0008 loss train: 0.4240 acc train: 0.9500 loss val: 0.7070 acc val: 0.8033
Epoch: 0009 loss train: 0.4328 acc train: 0.9429 loss val: 0.7047 acc val: 0.8067
Epoch: 0010 loss train: 0.4287 acc train: 0.9357 loss val: 0.7030 acc val: 0.8233
Epoch: 0011 loss train: 0.4207 acc train: 0.9429 loss val: 0.7022 acc val: 0.8200
Epoch: 0012 loss train: 0.4275 acc train: 0.9500 loss val: 0.7018 acc val: 0.8200
Epoch: 0013 loss

Epoch: 0115 loss train: 0.3166 acc train: 0.9429 loss val: 0.6617 acc val: 0.8133
Epoch: 0116 loss train: 0.3408 acc train: 0.9429 loss val: 0.6605 acc val: 0.8100
Epoch: 0117 loss train: 0.3055 acc train: 0.9643 loss val: 0.6587 acc val: 0.8067
Epoch: 0118 loss train: 0.2965 acc train: 0.9714 loss val: 0.6565 acc val: 0.8000
Epoch: 0119 loss train: 0.3115 acc train: 0.9571 loss val: 0.6545 acc val: 0.8100
Epoch: 0120 loss train: 0.3330 acc train: 0.9429 loss val: 0.6533 acc val: 0.8100
Epoch: 0121 loss train: 0.3069 acc train: 0.9357 loss val: 0.6521 acc val: 0.8067
Epoch: 0122 loss train: 0.3220 acc train: 0.9357 loss val: 0.6523 acc val: 0.8067
Epoch: 0123 loss train: 0.2925 acc train: 0.9571 loss val: 0.6529 acc val: 0.8067
Epoch: 0124 loss train: 0.3442 acc train: 0.9286 loss val: 0.6537 acc val: 0.8133
Epoch: 0125 loss train: 0.3598 acc train: 0.9571 loss val: 0.6539 acc val: 0.8133
Epoch: 0126 loss train: 0.3178 acc train: 0.9786 loss val: 0.6538 acc val: 0.8067
Epoch: 0127 loss

In [62]:
# Testing
test()

Test set results: loss= 0.6441 accuracy= 0.8260
